In [0]:
# Acceder al df_bronze desde la vista si fue registrada, o leer desde la misma ruta
df_bronze = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/FileStore/tables/base_prueba_bronze.csv")
)


In [0]:
from pyspark.sql.functions import col

# Crear df_silver con limpieza de registros inválidos
df_silver = df_bronze.filter(
    (col("Combinaciones").isNotNull()) &
    (col("Combinaciones") > 0) &
    (col("In Stock").isNotNull()) &
    (col("Dispersión").isNotNull()) &
    (col("Tiendas con Inventario").isNotNull()) &
    (col("Tiendas sin Inventario").isNotNull())
)

# Mostrar resultado
display(df_silver)


In [0]:
df_silver.createOrReplaceTempView("silver_base_prueba")


## Estadisticas Globales

In [0]:
# Estadísticas básicas globales
df_silver.describe(["Combinaciones", "Oust", "In Stock", "Dispersión"]).display()


## Promedio de In Stock y Dispersión por País

In [0]:
df_silver.groupBy("Pais").agg(
    {"In Stock": "avg", "Dispersión": "avg"}
).withColumnRenamed("avg(In Stock)", "In_Stock_Promedio") \
 .withColumnRenamed("avg(Dispersión)", "Dispersión_Promedio") \
 .orderBy("In_Stock_Promedio", ascending=False) \
 .display()


## Promedio por Formato

In [0]:
df_silver.groupBy("Formato").agg(
    {"In Stock": "avg", "Dispersión": "avg"}
).withColumnRenamed("avg(In Stock)", "In_Stock_Promedio") \
 .withColumnRenamed("avg(Dispersión)", "Dispersión_Promedio") \
 .orderBy("In_Stock_Promedio", ascending=False) \
 .display()


## Promedio por Categoría (top 10 con peor In Stock)![](path)

In [0]:
df_silver.groupBy("Categoría").agg(
    {"In Stock": "avg"}
).withColumnRenamed("avg(In Stock)", "In_Stock_Promedio") \
 .orderBy("In_Stock_Promedio") \
 .limit(10) \
 .display()


## Visualización: In Stock Promedio por País (gráfico de barras)

In [0]:
import matplotlib.pyplot as plt
import pandas as pd

# Convertir a Pandas para graficar
df_pais = df_silver.groupBy("Pais").agg({"In Stock": "avg"}).toPandas()
df_pais.columns = ["Pais", "In_Stock_Promedio"]

# Gráfico de barras
df_pais.sort_values("In_Stock_Promedio", ascending=False).plot(
    x="Pais", y="In_Stock_Promedio", kind="bar", legend=False, title="In Stock Promedio por País"
)
plt.ylabel("In Stock %")
plt.show()


##  Visualización: Dispersión promedio por Formato

In [0]:
df_formato = df_silver.groupBy("Formato").agg({"Dispersión": "avg"}).toPandas()
df_formato.columns = ["Formato", "Dispersión_Promedio"]

df_formato.sort_values("Dispersión_Promedio", ascending=False).plot(
    x="Formato", y="Dispersión_Promedio", kind="bar", color="orange", title="Dispersión Promedio por Formato"
)
plt.ylabel("Dispersión")
plt.show()


## Mini KPI: Categorías con peor In Stock (gráfico)

In [0]:
df_cat = df_silver.groupBy("Categoría").agg({"In Stock": "avg"}).toPandas()
df_cat.columns = ["Categoría", "In_Stock_Promedio"]

# Top 10 con peor In Stock
df_cat.sort_values("In_Stock_Promedio").head(10).plot(
    x="Categoría", y="In_Stock_Promedio", kind="barh", color="red", title="Top 10 Categorías con Peor In Stock"
)
plt.xlabel("In Stock %")
plt.show()


# Caso 2 – Análisis de In Stock



## ¿Qué puede interpretar de los datos?

El análisis de In Stock a nivel global y desagregado por país, formato y categoría revela que:

- El **In Stock promedio general es de 96.36%**, por debajo de la meta corporativa establecida en **97.5%**.
- **Ningún país o formato alcanza simultáneamente las metas de In Stock y Dispersión**, lo que evidencia un **desempeño desigual en la cadena de abastecimiento**.
- Las **categorías con peor desempeño se concentran en cosméticos (D59)** y productos accesorios, lo que sugiere un problema estructural en estas líneas.

---

##  Conclusiones clave

### A nivel país:
- **Guatemala (GT)** tiene el mejor In Stock (97.04%), pero falla en dispersión.
- **Costa Rica (CR)** presenta el peor In Stock (95.66%) y dispersión más alta.
- **Nicaragua (NI)** destaca en dispersión (7.29%) pero no alcanza la meta de cobertura.

### A nivel formato:
- **DISCOUNT** es el único formato que se acerca a ambas metas (96.77% In Stock y 5.37% Dispersión).
- **HYPERMARKET** tiene el peor desempeño general.

### A nivel categoría:
- Ninguna de las **10 categorías con peor In Stock** supera el 95%.
- 4 de las 10 peores categorías pertenecen a **COSMETICS D59**, evidenciando una línea crítica.

---

## Recomendaciones

### 1. Intervención estratégica en la categoría **COSMETICS D59**
- Justificación: Aparece repetidamente en el bottom 10 de In Stock.
- Acción: Revisión del catálogo activo, niveles de seguridad, y forecast por canal.

---

### 2. Replicar el modelo de operación de **formato DISCOUNT**
- Justificación: Es el único formato que cumple o se acerca a los KPIs.
- Acción: Benchmark operativo, prácticas logísticas, política de surtido y rotación.

---

### 3. Intervención regional en **Costa Rica y Honduras**
- Justificación: Ambos países presentan los peores indicadores combinados.
- Acción: Diagnóstico logístico, revisión de cobertura por tienda, revisión de última milla o almacenes intermedios.

---

### 4. Establecer un sistema de alertas para categorías < 95% In Stock
- Justificación: La detección anticipada permitiría tomar decisiones más rápidas y evitar quiebres.
- Acción: Implementar semáforos operativos conectados a dashboards semanales.

#  Caso 3 – Análisis de Dispersión

## ¿Qué puede interpretar de los datos?

- La **dispersión promedio global es de 12.37%**, lo cual está por **encima de la meta corporativa de 9.0%**.
- La **alta desviación estándar (20.24%)** sugiere que **la dispersión varía mucho entre regiones y formatos**, lo que refleja **falta de consistencia en la cobertura por tienda**.
- La dispersión elevada implica que **algunas tiendas están completamente desabastecidas**, mientras otras sí tienen inventario del mismo producto, lo que genera **experiencias desiguales para el cliente y pérdida de ventas**.

---

## Conclusiones clave

### A nivel país:
- **Nicaragua (NI)** es el **único país que cumple la meta de dispersión** (7.29%), lo cual indica una cobertura más homogénea.
- **Costa Rica (CR)** presenta la **mayor dispersión** (17.67%), señal de grandes diferencias entre tiendas abastecidas y sin inventario.
- **Honduras (HN)** y **Guatemala (GT)** también superan el umbral, aunque con menor severidad.

### A nivel formato:
- **DISCOUNT** destaca con una dispersión del 5.37%  (cumple).
- **HYPERMARKET** y **BODEGA** presentan los niveles más altos (16.28% y 14.55% respectivamente), mostrando alta concentración de quiebres en tiendas específicas.

### A nivel categoría:
- Aunque no se visualizó aún en detalle para dispersión por categoría, se presume que las **categorías que aparecen con bajo In Stock** también presentan **altos niveles de dispersión**, especialmente en cosméticos y accesorios.

---

## Recomendaciones

### 1. **Replicar buenas prácticas logísticas de Nicaragua y formato DISCOUNT**
- Justificación: Son los únicos que cumplen con la meta de dispersión.
- Acción: Documentar procesos logísticos, frecuencia de abastecimiento y métodos de reposición; escalarlo al resto de países y formatos.

---

### 2. **Rediseñar distribución en tiendas HYPERMARKET y BODEGA**
- Justificación: Ambos presentan las peores cifras de dispersión.
- Acción: Mejorar la segmentación de inventario, aumentar visibilidad de cobertura por tienda y revisar rutas logísticas.

---

### 3. **Implementar alertas tempranas para dispersión crítica (>15%)**
- Justificación: Dispersión alta implica quiebres silenciosos en puntos de venta.
- Acción: Automatizar alertas semanales en dashboard operativo para categorías/formato/país con mayor dispersión.

---

### 4. **Validar cobertura en tiendas de Costa Rica**
- Justificación: CR presenta la dispersión más alta entre los 5 países.
- Acción: Auditar cobertura real vs teórica por tienda, identificar causas (demanda, rotura de stock, errores en picking).

# Caso 4 - SQL ?

# Caso 5 - Modelo de almacenamiento de datos


## Objetivo del modelo

Diseñar un modelo de almacenamiento robusto que permita:

- Construir un **historial confiable** de datos de inventario.
- Facilitar el **análisis de tendencias y evolución** en el tiempo.
- **Evitar duplicidades** en los registros.
- Permitir la **auditoría** del origen y carga de los datos.

---

## Aspectos considerados en el diseño

1. **Identificación única por fila**  
   - Se incluye un campo `id_inventario` para diferenciar cada registro.
   - Se combina con claves naturales como país, formato, categoría y mes.

2. **Fechas clave**  
   - `fecha_corte`: representa el momento al que pertenece el dato.
   - `fecha_carga`: marca cuándo fue ingresado al sistema.

3. **Trazabilidad**  
   - Campo `source` indica de dónde proviene el dato (por ejemplo, “Excel semanal”).
   - Se puede incluir un identificador de usuario o proceso si se automatiza.

4. **Control de duplicados**  
   - Uso de una **llave compuesta** lógica (`pais`, `formato`, `categoría`, `mes`, `fecha_corte`) para prevenir inserciones repetidas.
   - Validación previa a la ingesta para evitar sobrescribir historial existente.

5. **Extensibilidad**  
   - Puede crecer fácilmente en filas con nuevas semanas, meses o registros.
   - Permite construir dashboards, alertas o reportes sin rediseñar el esquema.

---

##  Estructura propuesta – Tabla `fact_inventario_historial`

| Campo                   | Tipo         | Descripción                                               |
|------------------------|--------------|-----------------------------------------------------------|
| `id_inventario`        | UUID / String| ID tecnico del registro                              |
| `pais`                 | String       | Código del país (ej.: CR, GT, HN)                         |
| `formato`              | String       | Tipo de tienda (DISCOUNT, BODEGA, etc.)                   |
| `categoría`            | String       | Categoría de producto                                     |
| `mes`                  | Integer      | Mes del dato (ej.: 1 a 12)                                |
| `combinaciones`        | Integer      | Total de combinaciones registradas                        |
| `oust`                 | Integer      | Total de quiebres (Oust)                                  |
| `tiendas_con_inventario` | Integer    | Tiendas que tenían el producto                            |
| `tiendas_sin_inventario` | Integer    | Tiendas que no tenían el producto                         |
| `in_stock`             | Decimal(5,4) | Porcentaje de In Stock                                    |
| `dispersión`           | Decimal(5,4) | Porcentaje de Dispersión                                  |
| `fecha_corte`          | Date         | Fecha a la que corresponde el dato                        |
| `fecha_carga`          | Timestamp    | Fecha en que se cargó el registro al sistema              |
| `source`               | String       | Fuente del dato (ej.: `upload_excel`, `pipeline_autom`)   |

---

## Justificación del modelo

- **Historial**: cada fila representa un snapshot en un momento del tiempo (`fecha_corte`), permitiendo análisis temporales.
- **Auditoría**: al registrar fuente y fecha de carga, se puede rastrear cuándo y cómo llegó cada dato.
- **Prevención de duplicados**: combinación de claves naturales + validación de existencia previa.
- **Escalabilidad**: puede adaptarse a más fuentes, más granularidad o más niveles de análisis.

---

## Usos y beneficios

- Permite construir dashboards de evolución del In Stock y Dispersión.
- Facilita la detección de tendencias por país, formato o categoría.
- Sirve como base para modelos de forecasting y alertas.
- Apoya la toma de decisiones logísticas basadas en históricos reales.

# Caso 6 – Modelo de almacenamiento bajo carga semanal desde Excel



## Escenario Completo

| Elemento | Definición |
|:---|:---|
| **Origen** | Archivos Excel subidos semanalmente por usuarios |
| **Ingesta** | Automatizada usando pipelines (ej. Azure Data Factory, Databricks Autoloader) |
| **Procesamiento** | Validaciones, transformación y carga de históricos |
| **Almacenamiento** | BigQuery en Google Cloud Platform |
| **Consumo** | Equipo de BI usando herramientas como Power BI, Looker o Data Studio |

---

# Flujo propuesto

1. **Usuario carga Excel** a un **bucket en cloud storage** (ej.: GCS bucket o Azure Blob Storage).

2. **Trigger automático** al detectar un archivo nuevo:
   - Puede ser un **Event Trigger** (ej.: Cloud Functions, Azure Functions).
   - Inicia pipeline de procesamiento.

3. **Pipeline de procesamiento** (ej.: Databricks o Dataflow):
   - Leer el archivo.
   - Validar formato, duplicados, tipos de datos.
   - Transformar y calcular si es necesario.
   - Enriquecer con metadata (`source`, `fecha_carga`, `usuario`).

4. **Carga en BigQuery** en la tabla `fact_inventario_historial`:
   - Modo **append** para mantener el historial.
   - Validaciones de unicidad por `pais`, `formato`, `categoría`, `fecha_corte`.

5. **Publicación para BI**:
   - BigQuery expone datasets certificados.
   - Equipo de BI conecta directamente usando Power BI, Looker o Google Data Studio para dashboards dinámicos.

---

#  Tecnologías

| Etapa | Tecnología recomendada |
|:---|:---|
| Carga de archivos | Google Cloud Storage (GCS) o Azure Blob Storage |
| Ingesta automatizada | Cloud Functions + Databricks Autoloader o Azure Data Factory |
| Procesamiento | Databricks (PySpark) o Google Dataflow |
| Almacenamiento final | BigQuery |
| Consumo BI | Power BI, Looker, Google Data Studio |

---

# **Flujo** :

```
Usuario
  ↓
Carga Excel → Bucket en Storage (GCS / Blob Storage)
  ↓
Trigger automático (Cloud Function / Azure Function)
  ↓
Pipeline de procesamiento (Databricks / Dataflow)
  ↓
Tabla BigQuery: fact_inventario_historial
  ↓
Dashboards BI (Power BI / Looker / Data Studio)
```

---

# Resumen de Beneficios de esta Arquitectura

| Aspecto | Cómo se cubre |
|:---|:---|
| Historial de datos | Cada carga es nueva y no borra el historial |
| Prevención de duplicados | Validaciones en pipeline antes de insertar |
| Auditoría | `fecha_corte`, `fecha_carga`, `source`, `usuario` disponibles |
| Disponibilidad para BI | BigQuery optimizado para consumo masivo |
| Automatización | Cero intervención manual una vez cargado el Excel |

---

# Conclusión

Con este enfoque, se lograra un **flujo real de data engineering** moderno, escalable, y totalmente preparado para crecimiento futuro y dashboards de BI.